# Optimisation problem
* Maximise sth with a set of constraints
* Computational challenging, but can find a pretty good approximate solution


# Knapsack problem
* 2 variances
    * 0-1
    * continuous or fractional (considerably easier)


## Question statement
* Each item is represented by a pair, <value, weight>
* The knapsack can accommodate items with a total weight of no more than w
* A vector, I, of length n, represents the set of available items. Each element of the vector is an item
* A vector, V, of length n, is used to indicate whether or not items are taken. If V[i] = 1, item I[i]is taken. If V[i] = 0, item I[i]is not taken
* Find a V that maximise $$\sum_{i=0}^{n-1} V[i]*I[i].value$$ subject to the constraint that $$\sum_{i=0}^{n-1} V[i]*I[i].weight <= w$$ 

In [1]:
class Food(object):
    def __init__(self, n, v, w):
        self.name = n
        self.value = v
        self.calories = w
    def getValue(self):
        return self.value
    def getCost(self):
        return self.calories
    def density(self):
        return self.getValue()/self.getCost()
    def __str__(self):
        return self.name + ': <' + str(self.value)\
                 + ', ' + str(self.calories) + '>'

def buildMenu(names, values, calories):
    """names, values, calories lists of same length.
       name a list of strings
       values and calories lists of numbers
       returns list of Foods"""
    menu = []
    for i in range(len(values)):
        menu.append(Food(names[i], values[i],
                          calories[i]))
    return menu

names = ['wine', 'beer', 'pizza', 'burger', 'fries',
         'cola', 'apple', 'donut', 'cake']
values = [89,90,95,100,90,79,50,10]
calories = [123,154,258,354,365,150,95,195]
foods = buildMenu(names, values, calories)

## Bruteforce algorithm
1. Generate powerset
2. Remove all combinations whose total units exceeds the allowed weight
3. From the remaining, choose any one whose value is the greatest
O(2^n): note that 0/1 knapsack is inherently exponential

In [2]:
def powerSet(items):
    N = len(items)
    # enumerate the 2**N possible combinations
    for i in range(2**N):
        combo = []
        for j in range(N):
            # test bit jth of integer i
            if (i >> j) % 2 == 1:
                combo.append(items[j])
        yield combo

## Greedy algorithm - a practical alternative
* metric: most valuable/lightest/highest value per unit
* Pros
    * Easy to implement
    * computationally efficient: O(nlogn) + O(n) = O(nlogn)
* Con: may not give the best answer - only give a local max

In [3]:
def greedy(items, maxCost, keyFunction):
    """Assumes items a list, maxCost >= 0,
         keyFunction maps elements of items to numbers"""
    itemsCopy = sorted(items, key = keyFunction,
                       reverse = True)  # O(nlogn) #sorted: return a copy, don't want a side effect on the parameter
    result = []
    totalValue, totalCost = 0.0, 0.0
    for i in range(len(itemsCopy)):     #O(n)
        if (totalCost+itemsCopy[i].getCost()) <= maxCost:
            result.append(itemsCopy[i])
            totalCost += itemsCopy[i].getCost()
            totalValue += itemsCopy[i].getValue()
    return (result, totalValue)

def testGreedy(items, constraint, keyFunction):
    taken, val = greedy(items, constraint, keyFunction)
    print('Total value of items taken =', val)
    for item in taken:
        print('   ', item)

def testGreedys(foods, maxUnits):
    print('Use greedy by value to allocate', maxUnits,
          'calories')
    testGreedy(foods, maxUnits, Food.getValue)
    print('\nUse greedy by cost to allocate', maxUnits,
          'calories')
    testGreedy(foods, maxUnits,
               lambda x: 1/Food.getCost(x))
    print('\nUse greedy by density to allocate', maxUnits,
          'calories')
    testGreedy(foods, maxUnits, Food.density)


names = ['wine', 'beer', 'pizza', 'burger', 'fries',
         'cola', 'apple', 'donut', 'cake']
values = [89,90,95,100,90,79,50,10]
calories = [123,154,258,354,365,150,95,195]
foods = buildMenu(names, values, calories)
testGreedys(foods, 750)

Use greedy by value to allocate 750 calories
Total value of items taken = 284.0
    burger: <100, 354>
    pizza: <95, 258>
    wine: <89, 123>

Use greedy by cost to allocate 750 calories
Total value of items taken = 318.0
    apple: <50, 95>
    wine: <89, 123>
    cola: <79, 150>
    beer: <90, 154>
    donut: <10, 195>

Use greedy by density to allocate 750 calories
Total value of items taken = 318.0
    wine: <89, 123>
    beer: <90, 154>
    cola: <79, 150>
    apple: <50, 95>
    donut: <10, 195>


## Search-Tree implementation
* Left-first, depth first enumeration
    * The first element is selected from the still to be considered items
        * If there is room for that item in the knapsack, a node is constructed that reflects the consequence of choosing to take that item. By convention, we draw that as the left child
        * We also explore the consequences of not taking that item. This is the right child
    * The process is then applied recursivelyto non-leaf children
    * Finally, chose a node with the highest value that meets constraints
* Complexity
    * no. nodes at level i = 2^i
    * so O(2^(n+1)), but an obvious optimization: don’t explore parts of tree that violate constraint

In [4]:
def maxVal(toConsider, avail):
    """Assumes toConsider a list of items, avail a weight
       Returns a tuple of the total value of a solution to the
         0/1 knapsack problem and the items of that solution"""
    if toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getCost() > avail:
        #Explore right branch only
        result = maxVal(toConsider[1:], avail)
    else:
        nextItem = toConsider[0]
        #Explore left branch
        withVal, withToTake = maxVal(toConsider[1:],
                                     avail - nextItem.getCost())
        withVal += nextItem.getValue()
        #Explore right branch
        withoutVal, withoutToTake = maxVal(toConsider[1:], avail)
        #Choose better branch
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    return result

def testMaxVal(foods, maxUnits, printItems = True):
    print('Use search tree to allocate', maxUnits,
          'calories')
    val, taken = maxVal(foods, maxUnits)
    print('Total value of items taken =', val)
    if printItems:
        for item in taken:
            print('   ', item)

testMaxVal(foods, 750)


Use search tree to allocate 750 calories
Total value of items taken = 353
    cola: <79, 150>
    pizza: <95, 258>
    beer: <90, 154>
    wine: <89, 123>


## Dynamic programming
* When does it work?
    * Optimal substructure: a globally optimal solution can be found by combining optimal solutions to local subproblems
    * Overlapping subproblems: finding an optimal solution involves solving the same problem multiple times
* in the knapsack search tree, each node = <taken, left, value, remaining calories>, but as long as "left" and "remaining calories" are the same, it is the same problem
* although time complexity is still O(2^n), it often yields good performance for a subclass of optimization problems—those with optimal substructure and overlapping subproblems
    * Solution always correct
    * Fast under the right circumstances

In [5]:
def fastFib(n, memo = {}):
    """Assumes n is an int >= 0, memo used only by recursive calls
       Returns Fibonacci of n"""
    if n == 0 or n == 1:
        return 1
    try:
        return memo[n]
    except KeyError:
        result = fastFib(n-1, memo) + fastFib(n-2, memo)
        memo[n] = result
        return result

for i in range(121):
    print('fib(' + str(i) + ') =', fastFib(i))

fib(0) = 1
fib(1) = 1
fib(2) = 2
fib(3) = 3
fib(4) = 5
fib(5) = 8
fib(6) = 13
fib(7) = 21
fib(8) = 34
fib(9) = 55
fib(10) = 89
fib(11) = 144
fib(12) = 233
fib(13) = 377
fib(14) = 610
fib(15) = 987
fib(16) = 1597
fib(17) = 2584
fib(18) = 4181
fib(19) = 6765
fib(20) = 10946
fib(21) = 17711
fib(22) = 28657
fib(23) = 46368
fib(24) = 75025
fib(25) = 121393
fib(26) = 196418
fib(27) = 317811
fib(28) = 514229
fib(29) = 832040
fib(30) = 1346269
fib(31) = 2178309
fib(32) = 3524578
fib(33) = 5702887
fib(34) = 9227465
fib(35) = 14930352
fib(36) = 24157817
fib(37) = 39088169
fib(38) = 63245986
fib(39) = 102334155
fib(40) = 165580141
fib(41) = 267914296
fib(42) = 433494437
fib(43) = 701408733
fib(44) = 1134903170
fib(45) = 1836311903
fib(46) = 2971215073
fib(47) = 4807526976
fib(48) = 7778742049
fib(49) = 12586269025
fib(50) = 20365011074
fib(51) = 32951280099
fib(52) = 53316291173
fib(53) = 86267571272
fib(54) = 139583862445
fib(55) = 225851433717
fib(56) = 365435296162
fib(57) = 591286729879
fib(5

In [6]:
import random
def fastMaxVal(toConsider, avail, memo = {}):
    """Assumes toConsider a list of subjects, avail a weight
         memo supplied by recursive calls
       Returns a tuple of the total value of a solution to the 0/1 knapsack problem and the subjects of that solution
       Memo is a dict with key being a tuple(#items left to be considered, available weight)"""
    if (len(toConsider), avail) in memo:                #check whether it is already solved
        result = memo[(len(toConsider), avail)]
    elif toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getCost() > avail:
        #Explore right branch only
        result = fastMaxVal(toConsider[1:], avail, memo)
    else:
        nextItem = toConsider[0]
        #Explore left branch
        withVal, withToTake =\
                 fastMaxVal(toConsider[1:],
                            avail - nextItem.getCost(), memo)
        withVal += nextItem.getValue()
        #Explore right branch
        withoutVal, withoutToTake = fastMaxVal(toConsider[1:],
                                                avail, memo)
        #Choose better branch
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    memo[(len(toConsider), avail)] = result              # update memo
    return result

def testMaxVal(foods, maxUnits, algorithm, printItems = True):
    print('Menu contains', len(foods), 'items')
    print('Use search tree to allocate', maxUnits,
          'calories')
    val, taken = algorithm(foods, maxUnits)
    if printItems:
        print('Total value of items taken =', val)
        for item in taken:
            print('   ', item)
            
def buildLargeMenu(numItems, maxVal, maxCost):
    items = []
    for i in range(numItems):
        items.append(Food(str(i),
                          random.randint(1, maxVal),
                          random.randint(1, maxCost)))
    return items

for numItems in (2, 4, 8, 16, 32, 64, 128, 256, 512, 1024):
    numCalls = 0
    items = buildLargeMenu(numItems, 90, 250)
    testMaxVal(items, 750, fastMaxVal)

Menu contains 2 items
Use search tree to allocate 750 calories
Total value of items taken = 79
    1: <20, 20>
    0: <59, 95>
Menu contains 4 items
Use search tree to allocate 750 calories
Total value of items taken = 161
    3: <70, 224>
    2: <64, 214>
    0: <27, 211>
Menu contains 8 items
Use search tree to allocate 750 calories
Total value of items taken = 375
    7: <61, 132>
    6: <87, 52>
    5: <65, 156>
    3: <75, 182>
    1: <87, 142>
Menu contains 16 items
Use search tree to allocate 750 calories
Total value of items taken = 421
    14: <70, 98>
    11: <12, 18>
    8: <60, 134>
    7: <90, 139>
    6: <69, 159>
    3: <65, 125>
    1: <55, 72>
Menu contains 32 items
Use search tree to allocate 750 calories
Total value of items taken = 705
    14: <70, 98>
    13: <89, 189>
    28: <81, 23>
    23: <85, 18>
    21: <74, 28>
    19: <44, 48>
    18: <46, 65>
    13: <64, 74>
    1: <84, 158>
    0: <68, 35>
Menu contains 64 items
Use search tree to allocate 750 calories


In [7]:
def countingFastMaxVal(toConsider, avail, memo = {}):
    """Assumes toConsider a list of subjects, avail a weight
         memo supplied by recursive calls
       Returns a tuple of the total value of a solution to the
         0/1 knapsack problem and the subjects of that solution"""
    global numCalls
    numCalls += 1
    
    if (len(toConsider), avail) in memo:
        result = memo[(len(toConsider), avail)]
    elif toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getCost() > avail:
        #Explore right branch only
        result = countingFastMaxVal(toConsider[1:], avail, memo)
    else:
        nextItem = toConsider[0]
        #Explore left branch
        withVal, withToTake =\
                 countingFastMaxVal(toConsider[1:],
                            avail - nextItem.getCost(), memo)
        withVal += nextItem.getValue()
        #Explore right branch
        withoutVal, withoutToTake = countingFastMaxVal(toConsider[1:],
                                                avail, memo)
        #Choose better branch
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    memo[(len(toConsider), avail)] = result
    return result

for numItems in (2, 4, 8, 16, 32, 64, 128, 256, 512, 1024):
    numCalls = 0
    items = buildLargeMenu(numItems, 90, 250)
    testMaxVal(items, 750, countingFastMaxVal, False)
    print('Number of calls =', numCalls)

Menu contains 2 items
Use search tree to allocate 750 calories
Number of calls = 7
Menu contains 4 items
Use search tree to allocate 750 calories
Number of calls = 25
Menu contains 8 items
Use search tree to allocate 750 calories
Number of calls = 455
Menu contains 16 items
Use search tree to allocate 750 calories
Number of calls = 5818
Menu contains 32 items
Use search tree to allocate 750 calories
Number of calls = 20932
Menu contains 64 items
Use search tree to allocate 750 calories
Number of calls = 44606
Menu contains 128 items
Use search tree to allocate 750 calories
Number of calls = 74239
Menu contains 256 items
Use search tree to allocate 750 calories
Number of calls = 188072
Menu contains 512 items
Use search tree to allocate 750 calories
Number of calls = 342430
Menu contains 1024 items
Use search tree to allocate 750 calories
Number of calls = 710295


# Graph theory

## Graph
* set of nodes
* set of edges
    * undirected (graph) or directed (digraph): source and destination nodes
    * unweighted or weighted
* Tree: a special graph
    * any pair of nodes is connected by a single path
* Questions
    * Shortest path problem
    * graph partition problem
    * min-cut max-flow problem

## Define a graph
* Adjacency matrix
    * Rows: source nodes
    * Columns: destination nodes
    * Cell[s, d] = 1 if there is an edge from s to d; 0 otherwise
* Adjacency list
    * Associate with each node a list of destination nodes

In [8]:
class Node(object):
    def __init__(self, name):
        """Assumes name is a string"""
        self.name = name
    def getName(self):
        return self.name
    def __str__(self):
        return self.name

class Edge(object):
    def __init__(self, src, dest):
        """Assumes src and dest are nodes"""
        self.src = src
        self.dest = dest
    def getSource(self):
        return self.src
    def getDestination(self):
        return self.dest
    def __str__(self):
        return self.src.getName() + '->' + self.dest.getName()

In [9]:
class Digraph(object):
    """edges is a dict mapping each node to a list of its children"""
    def __init__(self):
        self.edges = {}
    def addNode(self, node):
        if node in self.edges:
            raise ValueError('Duplicate node')
        else:
            self.edges[node] = []
    def addEdge(self, edge):
        src = edge.getSource()
        dest = edge.getDestination()
        if not (src in self.edges and dest in self.edges):
            raise ValueError('Node not in graph')
        self.edges[src].append(dest)
    def childrenOf(self, node):
        return self.edges[node]
    def hasNode(self, node):
        return node in self.edges
    def getNode(self, name):
        for n in self.edges:
            if n.getName() == name:
                return n
        raise NameError(name)
    def __str__(self):
        result = ''
        for src in self.edges:
            for dest in self.edges[src]:
                result = result + src.getName() + '->'\
                         + dest.getName() + '\n'
        return result[:-1] #omit final newline

In [10]:
"""
Graph is a subclass of digraph because:
If client code works correctly using an instance of the supertype, it should also work correctly when an instance of the subtype is 
substituted for the instance of the supertype
i.e. Any program that works with a Digraph will also work with a Graph (but not vice versa)
"""
class Graph(Digraph):  
    def addEdge(self, edge):    # use this version instead
        Digraph.addEdge(self, edge)
        rev = Edge(edge.getDestination(), edge.getSource())
        Digraph.addEdge(self, rev)

In [11]:
def buildCityGraph(graphType):
    g = graphType()
    for name in ('Boston', 'Providence', 'New York', 'Chicago',
                 'Denver', 'Phoenix', 'Los Angeles'): #Create 7 nodes
        g.addNode(Node(name))
    g.addEdge(Edge(g.getNode('Boston'), g.getNode('Providence')))
    g.addEdge(Edge(g.getNode('Boston'), g.getNode('New York')))
    g.addEdge(Edge(g.getNode('Providence'), g.getNode('Boston')))
    g.addEdge(Edge(g.getNode('Providence'), g.getNode('New York')))
    g.addEdge(Edge(g.getNode('New York'), g.getNode('Chicago')))
    g.addEdge(Edge(g.getNode('Chicago'), g.getNode('Denver')))
    g.addEdge(Edge(g.getNode('Denver'), g.getNode('Phoenix')))
    g.addEdge(Edge(g.getNode('Denver'), g.getNode('New York')))
    g.addEdge(Edge(g.getNode('Los Angeles'), g.getNode('Boston')))
    return g
    
print(buildCityGraph(Graph))

Boston->Providence
Boston->New York
Boston->Providence
Boston->Los Angeles
Providence->Boston
Providence->Boston
Providence->New York
New York->Boston
New York->Providence
New York->Chicago
New York->Denver
Chicago->New York
Chicago->Denver
Denver->Chicago
Denver->Phoenix
Denver->New York
Phoenix->Denver
Los Angeles->Boston


## Shortest path problems
* Shortest path from n1 to n2
    * Shortest sequence of edges such that:
        * Source node of first edge is n1
        * Destination of last edge is n2
    * For edges, e1 and e2, in the sequence, if e2 follows e1 in the sequence, the source of e2 is the destination of e1
* Shortest weighted path
    * Minimize the sum of the weights of the edges in the path

### Depth-first search
* Note: graph might have cycles, so we must keep track of what nodes we have visited
* start at an intial node, consider all edges that leave that node in some order
* follow the first edge, check if at goal node.
* if not, repeat the process from new node.
* continue until either find goal node or run out of options.
    * when run out of options, backtrack to previous nodes and try the next edge

In [15]:
def printPath(path):
    """Assumes path is a list of nodes"""
    result = ''
    for i in range(len(path)):
        result = result + str(path[i])
        if i != len(path) - 1:
            result = result + '->'
    return result 

def DFS(graph, start, end, path, shortest, toPrint = False):
    """Assumes graph is a Digraph; start and end are nodes;
          path and shortest are lists of nodes
       Returns a shortest path from start to end in graph"""
    path = path + [start]
    if toPrint:
        print('Current DFS path:', printPath(path))
    if start == end:
        return path
    for node in graph.childrenOf(start):
        if node not in path: #avoid cycles
            if shortest == None or len(path) < len(shortest):
                newPath = DFS(graph, node, end, path, shortest,
                              toPrint)
                if newPath != None:
                    shortest = newPath
        elif toPrint:
            print('Already visited', node)
    print("shortest", printPath(shortest))
    return shortest
    
def shortestPath(graph, start, end, toPrint = False):
    """Assumes graph is a Digraph; start and end are nodes
       Returns a shortest path from start to end in graph"""
    return DFS(graph, start, end, [], None, toPrint)

def testSP(source, destination):
    g = buildCityGraph(Digraph)
    sp = shortestPath(g, g.getNode(source), g.getNode(destination),
                      toPrint = True)
    if sp != None:
        print('Shortest path from', source, 'to',
              destination, 'is', printPath(sp))
    else:
        print('There is no path from', source, 'to', destination)

#testSP('Chicago', 'Boston')
testSP('Boston', 'Phoenix')

Current DFS path: Boston
Current DFS path: Boston->Providence
Already visited Boston
Current DFS path: Boston->Providence->New York
Current DFS path: Boston->Providence->New York->Chicago
Current DFS path: Boston->Providence->New York->Chicago->Denver
Current DFS path: Boston->Providence->New York->Chicago->Denver->Phoenix
Already visited New York
shortest Boston->Providence->New York->Chicago->Denver->Phoenix
shortest Boston->Providence->New York->Chicago->Denver->Phoenix
shortest Boston->Providence->New York->Chicago->Denver->Phoenix
shortest Boston->Providence->New York->Chicago->Denver->Phoenix
Current DFS path: Boston->New York
Current DFS path: Boston->New York->Chicago
Current DFS path: Boston->New York->Chicago->Denver
Current DFS path: Boston->New York->Chicago->Denver->Phoenix
Already visited New York
shortest Boston->New York->Chicago->Denver->Phoenix
shortest Boston->New York->Chicago->Denver->Phoenix
shortest Boston->New York->Chicago->Denver->Phoenix
shortest Boston->New 

### Breadth-first search
* Note: cannot be easily modified to solve a weighted shortest path problem
* start at an intial node, consider all edges that leave that node in some order
* follow the first edge, check if at goal node.
* if not, try the next edge from the current node.
* continue until either find goal node or run out of options.
    * when run out of edge options, move the next node at same distance from start, repeat
    * when run out of node options, move to next level in the graph, repeat

In [13]:
def BFS(graph, start, end, toPrint = False, printQueue = False):
    """Assumes graph is a Digraph; start and end are nodes
       Returns a shortest path from start to end in graph"""
    initPath = [start]    # path is a list of nodes
    pathQueue = [initPath]   # a list of paths
    while len(pathQueue) != 0:
        if printQueue:
            print('Queue ', len(pathQueue))
            for p in pathQueue:
                print(printPath(p))
        #Get and remove oldest element in pathQueue
        tmpPath = pathQueue.pop(0)
        if toPrint:
            print('Current BFS path:', printPath(tmpPath), '\n')
        lastNode = tmpPath[-1]
        if lastNode == end:
            return tmpPath
        for nextNode in graph.childrenOf(lastNode):
            if nextNode not in tmpPath:
                newPath = tmpPath + [nextNode]
                pathQueue.append(newPath)
    return None

def shortestPath(graph, start, end, toPrint = False, printQueue = True):
    """Assumes graph is a Digraph; start and end are nodes
       Returns a shortest path from start to end in graph"""
    return BFS(graph, start, end, toPrint, printQueue)
    
testSP('Boston', 'Phoenix')

Queue  1
Boston
Current BFS path: Boston 

Queue  2
Boston->Providence
Boston->New York
Current BFS path: Boston->Providence 

Queue  2
Boston->New York
Boston->Providence->New York
Current BFS path: Boston->New York 

Queue  2
Boston->Providence->New York
Boston->New York->Chicago
Current BFS path: Boston->Providence->New York 

Queue  2
Boston->New York->Chicago
Boston->Providence->New York->Chicago
Current BFS path: Boston->New York->Chicago 

Queue  2
Boston->Providence->New York->Chicago
Boston->New York->Chicago->Denver
Current BFS path: Boston->Providence->New York->Chicago 

Queue  2
Boston->New York->Chicago->Denver
Boston->Providence->New York->Chicago->Denver
Current BFS path: Boston->New York->Chicago->Denver 

Queue  2
Boston->Providence->New York->Chicago->Denver
Boston->New York->Chicago->Denver->Phoenix
Current BFS path: Boston->Providence->New York->Chicago->Denver 

Queue  2
Boston->New York->Chicago->Denver->Phoenix
Boston->Providence->New York->Chicago->Denver->Phoe